In [1]:
#Import das bibliotecas
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import math

In [2]:
# Criação do dicionário que receberá os registros
dic_produtos = {'produto':[], 'preco':[], 'desconto':[], 'link':[]}

# Define os headers que serão utilizados no programa
headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"}

In [3]:
#FutFanatics - Parte 1
url = 'https://www.futfanatics.com.br/loja/catalogo.php?loja=311840&categoria=1161&variacao=tamanho_41&range=&order=2&pg=1'

site = requests.get(url, headers=headers)

encoding = site.encoding if 'charset' in site.headers.get('content-type', '').lower() else None
parser = 'html5lib'
soup = BeautifulSoup(site.content, parser, from_encoding=encoding)

totais = soup.find_all('div', class_="search-counter")

total_texto = totais[0].get_text().strip()

index = total_texto.find(' ')

qtd  = total_texto[:index]

ultima_pagina = math.ceil(int(qtd)/40)

print("Quantidade de produtos (FutFanatics):",qtd)
print("Quantidade de páginas (FutFanatics):",ultima_pagina)

Quantidade de produtos (FutFanatics): 640
Quantidade de páginas (FutFanatics): 16


In [4]:
#FutFanatics - Parte 2
for i in range(1, ultima_pagina+1):
    print('FutFanatics - página', i, 'de', ultima_pagina)
    url_pag = f'https://www.futfanatics.com.br/loja/catalogo.php?loja=311840&categoria=1161&variacao=tamanho_41&range=&order=2&pg={i}'
    site = requests.get(url_pag, headers=headers)
    soup = BeautifulSoup(site.content, parser, from_encoding=encoding)
    produtos = soup.find_all('div', class_=re.compile('product-item'))

    for produto in produtos:
        prod = produto.find('div', class_=re.compile('product-name')).get_text().strip()
        preco = produto.find('div', class_='price').get_text().strip()
        desconto = 0
        descontos = produto.find_all('span', class_=re.compile('discount'))
        if len(descontos) > 0:
            desconto = descontos[0].get_text().strip()
        link = produto.find_all('a', href=True)[0]['href']
        dic_produtos['produto'].append(prod)
        dic_produtos['preco'].append(preco)
        dic_produtos['desconto'].append(desconto)
        dic_produtos['link'].append(link)

FutFanatics - página 1 de 16
FutFanatics - página 2 de 16
FutFanatics - página 3 de 16
FutFanatics - página 4 de 16
FutFanatics - página 5 de 16
FutFanatics - página 6 de 16
FutFanatics - página 7 de 16
FutFanatics - página 8 de 16
FutFanatics - página 9 de 16
FutFanatics - página 10 de 16
FutFanatics - página 11 de 16
FutFanatics - página 12 de 16
FutFanatics - página 13 de 16
FutFanatics - página 14 de 16
FutFanatics - página 15 de 16
FutFanatics - página 16 de 16


In [5]:
# Cria o DataFrame Pandas com os produtos capturados
df = pd.DataFrame(dic_produtos)
df.head()

,produto,preco,desconto,link
0,Chuteira Mizuno Morelia Neo IV Beta Japan Camp...,"R$2.099,90",16%,https://www.futfanatics.com.br/chuteira-mizuno...
1,Chuteira Mizuno Morelia Neo IV Japan Campo Ver...,"R$1.749,90",12%,https://www.futfanatics.com.br/chuteira-mizuno...
2,Chuteira Mizuno Alfa Japan Campo Vermelha,"R$1.749,90",12%,https://www.futfanatics.com.br/chuteira-mizuno...
3,Chuteira Puma Future Ultimate FG/AG Campo Azul...,"R$1.499,90",0,https://www.futfanatics.com.br/chuteira-puma-f...
4,Chuteira Puma Future Ultimate FG/AG Campo Bran...,"R$1.299,90",13%,https://www.futfanatics.com.br/chuteira-puma-f...


In [6]:
# Faz algumas transformações nas colunas capturadas
tabela = df.copy()
tabela['preco'] = tabela['preco'].apply(lambda x: str(x).replace(".",'').replace("R$",''))
tabela['preco'] = pd.to_numeric(tabela['preco'].apply(lambda x: str(x).replace(",",'.')))
tabela['desconto'] = pd.to_numeric(tabela['desconto'].apply(lambda x: str(x).replace("%",'')))
tabela['marca'] = tabela['produto'].astype(str).str.split().str[1]
tabela = tabela.sort_values(by=['desconto'],ascending=False)
tabela.head(15)

,produto,preco,desconto,link,marca
33,Chuteira Umbro Tocco II Pro FG Campo Preta,579.9,61,https://www.futfanatics.com.br/chuteira-umbro-...,Umbro
635,Chuteira Oxn Velox 4 Neo Campo Marinho e Rosa,69.9,56,https://www.futfanatics.com.br/chuteira-oxn-ve...,Oxn
637,Chuteira Oxn Velox 4 Campo Laranja,69.9,53,https://www.futfanatics.com.br/chuteira-oxn-ve...,Oxn
558,Chuteira Umbro Adamant League Futsal Preta e B...,99.9,50,https://www.futfanatics.com.br/chuteira-umbro-...,Umbro
621,Chuteira Oxn Velox 4 Neo Campo Preta e Laranja,79.9,50,https://www.futfanatics.com.br/chuteira-oxn-ve...,Oxn
555,Chuteira Umbro Stride Society Marinho e Azul,99.9,50,https://www.futfanatics.com.br/chuteira-umbro-...,Umbro
556,Chuteira Umbro Adamant League Society Azul e B...,99.9,50,https://www.futfanatics.com.br/chuteira-umbro-...,Umbro
623,Chuteira Oxn Velox 4 Neo Campo Preta e Azul,79.9,50,https://www.futfanatics.com.br/chuteira-oxn-ve...,Oxn
566,Chuteira Umbro Fifty IV Futsal Azul,99.9,50,https://www.futfanatics.com.br/chuteira-umbro-...,Umbro
636,Chuteira Oxn Avanti 2 Campo Preta,69.9,50,https://www.futfanatics.com.br/chuteira-oxn-av...,Oxn


In [7]:
# Exporta o resultado para CSV
tabela.to_csv('precoChuteiras.csv', encoding='utf-8', sep=',', index=False)